# Preparing structured data for SQLite : Oscars ceremonies

In [24]:
import pandas as pd

In [25]:
#Read csv
academy_awards = pd.read_csv('academy_awards.csv', encoding = "ISO-8859-1")

In [26]:
#Print the first rows
academy_awards.head()

,Year,Category,Nominee,Additional Info,Won?,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,2010 (83rd),Actor -- Leading Role,Javier Bardem,Biutiful {'Uxbal'},NO,NaN,NaN,NaN,NaN,NaN,NaN
1,2010 (83rd),Actor -- Leading Role,Jeff Bridges,True Grit {'Rooster Cogburn'},NO,NaN,NaN,NaN,NaN,NaN,NaN
2,2010 (83rd),Actor -- Leading Role,Jesse Eisenberg,The Social Network {'Mark Zuckerberg'},NO,NaN,NaN,NaN,NaN,NaN,NaN
3,2010 (83rd),Actor -- Leading Role,Colin Firth,The King's Speech {'King George VI'},YES,NaN,NaN,NaN,NaN,NaN,NaN
4,2010 (83rd),Actor -- Leading Role,James Franco,127 Hours {'Aron Ralston'},NO,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
awards = academy_awards

In [28]:
awards["Unnamed: 5"].value_counts()

*                                                                                                               7
 error-prone measurements on sets. [Digital Imaging Technology]"                                                1
 resilience                                                                                                     1
 D.B. "Don" Keele and Mark E. Engebretson has resulted in the over 20-year dominance of constant-directivity    1
 discoverer of stars                                                                                            1
Name: Unnamed: 5, dtype: int64

In [29]:
awards["Unnamed: 6"].value_counts()

*                                                                   9
 sympathetic                                                        1
 flexibility and water resistance                                   1
 direct radiator bass style cinema loudspeaker systems. [Sound]"    1
Name: Unnamed: 6, dtype: int64

In [30]:
awards["Unnamed: 7"].value_counts()

*                                                     1
 while requiring no dangerous solvents. [Systems]"    1
 kindly                                               1
Name: Unnamed: 7, dtype: int64

In [31]:
awards["Unnamed: 8"].value_counts()

*                                                 1
 understanding comedy genius - Mack Sennett.""    1
Name: Unnamed: 8, dtype: int64

In [32]:
#Get the year of the ceremony only for each row
awards["Year"] = awards["Year"].str[0:4]

In [33]:
#Convert the year to integers
awards["Year"] = awards["Year"].astype(int)

In [34]:
#Focus on post-2000 ceremonies
later_than_2000 = awards[awards["Year"] > 2000]

In [35]:
#Focus on the award categories listed in award_categories
award_categories = ["Actor -- Leading Role", "Actor -- Supporting Role", "Actress -- Leading Role", "Actress -- Supporting Role"]
nominations = later_than_2000[later_than_2000["Category"].isin(award_categories)]

In [36]:
#SQL does not handle text data very good. Thus, we want to convert "YES" or "NO" values to 0 or 1.
replace_dict = {"YES": 1, "NO": 0}
nominations["Won?"] = nominations["Won?"].map(replace_dict)

/Users/noulmi/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [37]:
#Convert the "WON?" column to "WON" for better reading
nominations["Won"] = nominations["Won?"]

/Users/noulmi/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [38]:
#Drop all the columns listed below

liste = ["Won?", 
"Unnamed: 5",
"Unnamed: 6",
"Unnamed: 7",
"Unnamed: 8",
"Unnamed: 9",
"Unnamed: 10"]

final_nominations = nominations.drop(liste, axis=1)

In [39]:
#Additionnal info contains the name of the movie and the name of the character in brackets. We want to separate these
#informations in two separate columns

final_nominations["additional_info_one"] = final_nominations["Additional Info"].str.rstrip("}")
final_nominations["additional_info_two"] = final_nominations["additional_info_one"].str.split(" {")
final_nominations["movies_name"] = final_nominations["additional_info_two"].str[0]
final_nominations["character"] = final_nominations["additional_info_two"].str[1]

In [40]:
#Dropper additionnal info columns

final_nominations = final_nominations.drop(["additional_info_one", "additional_info_two", "Additional Info"], axis=1)

In [41]:
#The movie's characters are in brackets. We use pandas' string method to replace brackets by blank spaces.

final_nominations["character"] = final_nominations["character"].str.replace("'", "")

In [42]:
import sqlite3 #import sqlite
conn = sqlite3.connect("nominations.db") #Create a connexion instance "conn" that is connected to our database
#nominations

In [47]:
#Create a nominations table from the final_nominations dataframe using our connexion to database.
#final_nominations.to_sql("nominations", conn, index=False)

In [44]:
#To verify our table was succesfully created, we print the table infos.

query = "pragma table_info(nominations)"
conn.execute(query).fetchall()

[(0, 'Year', 'INTEGER', 0, None, 0),
 (1, 'Category', 'TEXT', 0, None, 0),
 (2, 'Nominee', 'TEXT', 0, None, 0),
 (3, 'Won', 'INTEGER', 0, None, 0),
 (4, 'movies_name', 'TEXT', 0, None, 0),
 (5, 'character', 'TEXT', 0, None, 0)]

In [45]:
#AND we print the first 10 rows

query = '''SELECT* FROM nominations
LIMIT 10;'''
result = conn.execute(query).fetchall()

In [46]:
#Close the connexion to the database.

conn.close()